In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from collections import defaultdict

In [ ]:
# Load and display the image
image_path = 'data/20251106_201644.jpg'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis('off')
plt.title('Original Image')
plt.show()

In [ ]:
# Convert to HSV and show the color histogram
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

plt.figure(figsize=(10, 6))
histogram = cv2.calcHist([image_hsv], [0], None, [180], [0, 180])
bars = plt.bar(range(180), histogram.flatten(), width=1)

for i, bar in enumerate(bars):
    color = plt.cm.hsv(i / 180)
    bar.set_color(color)

plt.yscale('log')
plt.xlabel('Hue Value')
plt.ylabel('Frequency (log scale)')
plt.title('Hue Histogram with Rainbow Gradient (HSV)')
plt.xlim([0, 180])
plt.show()

In [ ]:
# Step 1: Detect white dots using adaptive thresholding
# Convert to grayscale
gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)

# Apply adaptive Gaussian thresholding
# blockSize: Size of pixel neighborhood (must be odd)
# C: Constant subtracted from weighted mean
white_mask = cv2.adaptiveThreshold(
    gray, 
    255, 
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    cv2.THRESH_BINARY, 
    blockSize=41,  # Easy to change: larger = more adaptive to lighting
    C=0            # Easy to change: lower = more sensitive
)

# Optional: Apply morphological operations to clean up the mask
kernel = np.ones((3, 3), np.uint8)
white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel, iterations=1)
white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel, iterations=1)

plt.figure(figsize=(12, 8), dpi=600)
plt.imshow(white_mask, cmap='gray')
plt.title('Step 1: White Dot Detection (Adaptive Gaussian Thresholding)')
plt.axis('off')
plt.show()

print(f"White pixels detected: {cv2.countNonZero(white_mask)}")

In [ ]:
# Step 2: Find contours and filter by purity and size
white_purity_threshold = 0.95  # Easy to change: 0.95 = 95%
min_dot_area = 10              # Easy to change: minimum area in pixels
max_dot_area = 100            # Easy to change: maximum area in pixels

# HSV range for white color
white_lower = np.array([0, 0, 180])
white_upper = np.array([180, 30, 255])

contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

def is_white_dot(hsv_image, contour, threshold=white_purity_threshold):
    """Check if a contour is actually a white dot by checking pixel purity."""
    contour_mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    cv2.drawContours(contour_mask, [contour], -1, 255, -1)
    
    total_pixels = cv2.countNonZero(contour_mask)
    if total_pixels == 0:
        return False, 0.0
    
    white_check_mask = cv2.inRange(hsv_image, white_lower, white_upper)
    white_pixels = cv2.countNonZero(cv2.bitwise_and(white_check_mask, contour_mask))
    purity = white_pixels / total_pixels
    
    return purity >= threshold, purity

# Filter contours and get dot centers
dot_centers = []
rejected_dots = []

for cnt in contours:
    area = cv2.contourArea(cnt)
    
    # Check size constraints
    if area < min_dot_area or area > max_dot_area:
        continue
    
    # Check white purity
    is_valid, purity = is_white_dot(image_hsv, cnt)
    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        
        if is_valid:
            dot_centers.append((cx, cy))
        else:
            rejected_dots.append((cx, cy, purity))

print(f"Total contours found: {len(contours)}")
print(f"Valid white dots (size: {min_dot_area}-{max_dot_area}, purity ≥ {white_purity_threshold}): {len(dot_centers)}")
print(f"Rejected dots: {len(rejected_dots)}")

# Visualize valid and rejected dots
vis_img = image_rgb.copy()
for cx, cy in dot_centers:
    cv2.circle(vis_img, (cx, cy), 8, (0, 255, 0), 2)  # Green for valid
for cx, cy, purity in rejected_dots:
    cv2.circle(vis_img, (cx, cy), 8, (255, 0, 0), 2)  # Red for rejected
    cv2.putText(vis_img, f"{purity:.2f}", (cx-20, cy-15), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

plt.figure(figsize=(12, 8), dpi=600)
plt.imshow(vis_img)
plt.title('Step 2: White Dot Filtering (Green=Valid, Red=Rejected)')
plt.axis('off')
plt.show()

In [ ]:
# Step 3: Create color masks
color_ranges = {
    "green": (np.array([40, 40, 40]), np.array([80, 255, 255])),
    "red": [
        (np.array([0, 40, 40]), np.array([10, 255, 255])),
        (np.array([170, 40, 40]), np.array([180, 255, 255])),
    ],
    "yellow": (np.array([20, 40, 40]), np.array([40, 255, 255])),
    "blue": (np.array([90, 40, 40]), np.array([130, 255, 255])),
    "pink": (np.array([140, 40, 40]), np.array([170, 255, 255])),
}

# Visualize all color masks
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (color, ranges) in enumerate(color_ranges.items()):
    if isinstance(ranges, list):
        mask = np.zeros(image_hsv.shape[:2], dtype=np.uint8)
        for lower, upper in ranges:
            mask |= cv2.inRange(image_hsv, lower, upper)
    else:
        lower, upper = ranges
        mask = cv2.inRange(image_hsv, lower, upper)
    
    kernel = np.ones((5, 5), np.uint8)
    mask_dilated = cv2.dilate(mask, kernel, iterations=2)
    
    axes[idx].imshow(mask_dilated, cmap='gray')
    axes[idx].set_title(f'{color.capitalize()} Mask (Dilated)')
    axes[idx].axis('off')

axes[-1].axis('off')
plt.suptitle('Step 3: Color Region Masks')
plt.tight_layout()
plt.show()

In [ ]:
# Step 4: Count dots by color
points = defaultdict(int)
dot_colors = {}

for color, ranges in color_ranges.items():
    if isinstance(ranges, list):
        mask = np.zeros(image_hsv.shape[:2], dtype=np.uint8)
        for lower, upper in ranges:
            mask |= cv2.inRange(image_hsv, lower, upper)
    else:
        lower, upper = ranges
        mask = cv2.inRange(image_hsv, lower, upper)
    
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=2)
    
    for cx, cy in dot_centers:
        if mask[cy, cx] > 0:
            points[color] += 1
            dot_colors[(cx, cy)] = color

print("Points by color:")
for color, count in sorted(points.items()):
    print(f"  {color.capitalize()}: {count}")
print(f"\nTotal: {sum(points.values())} points")

In [ ]:
# Step 5: Final visualization with colored circles
color_bgr_map = {
    "green": (0, 255, 0),
    "red": (255, 0, 0),
    "yellow": (255, 255, 0),
    "blue": (0, 0, 255),
    "pink": (255, 0, 255),
}

final_img = image_rgb.copy()

for (cx, cy), color in dot_colors.items():
    cv2.circle(final_img, (cx, cy), 10, color_bgr_map[color], 3)

# Add text overlay with scores
y_offset = 40
for color, count in sorted(points.items()):
    text = f"{color.capitalize()}: {count}"
    # Add black background for text
    cv2.rectangle(final_img, (5, y_offset-25), (200, y_offset+5), (0, 0, 0), -1)
    cv2.putText(final_img, text, (10, y_offset), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    y_offset += 40

# Add total
total_text = f"Total: {sum(points.values())}"
cv2.rectangle(final_img, (5, y_offset-25), (200, y_offset+5), (0, 0, 0), -1)
cv2.putText(final_img, total_text, (10, y_offset), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

plt.figure(figsize=(14, 10))
plt.imshow(final_img)
plt.title('Final Result: Counted Points by Color')
plt.axis('off')
plt.show()

# Save the result
cv2.imwrite('data/result.jpg', cv2.cvtColor(final_img, cv2.COLOR_RGB2BGR))
print("\nVisualization saved to data/result.jpg")